In [ ]:
%matplotlib inline

# Qualifying results overview

Plot the qualifying result with visualization the fastest times.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from timple.timedelta import strftimedelta
import fastf1
import fastf1.plotting
from fastf1.core import Laps


fastf1.Cache.enable_cache('../doc_cache')  # replace with your cache directory

# we only want support for timedelta plotting in this example
fastf1.plotting.setup_mpl(mpl_timedelta_support=True, color_scheme=None, misc_mpl_mods=False)

quali = fastf1.get_session(2021, 'Spanish Grand Prix', 'Q')
laps = quali.load_laps()

First, we need to get an array of all drivers.



In [ ]:
drivers = pd.unique(laps['Driver'])
print(drivers)

After that we'll get each drivers fastest lap, create a new laps object
from these laps, sort them by lap time and have pandas reindex them to
number them nicely by starting position.



In [ ]:
list_fastest_laps = list()
for drv in drivers:
    drvs_fastest_lap = laps.pick_driver(drv).pick_fastest()
    list_fastest_laps.append(drvs_fastest_lap)
fastest_laps = Laps(list_fastest_laps).sort_values(by='LapTime').reset_index(drop=True)

The plot is nicer to look at and more easily understandable if we just plot
the time differences. Therefore we subtract the fastest lap time from all
other lap times.



In [ ]:
pole_lap = fastest_laps.pick_fastest()
fastest_laps['LapTimeDelta'] = fastest_laps['LapTime'] - pole_lap['LapTime']

We can take a quick look at the laps we have to check if everything
looks all right. For this, we'll just check the 'Driver', 'LapTime'
and 'LapTimeDelta' columns.



In [ ]:
print(fastest_laps[['Driver', 'LapTime', 'LapTimeDelta']])

Finally, we'll create a list of team colors per lap to color our plot.



In [ ]:
team_colors = list()
for index, lap in fastest_laps.iterlaps():
    color = fastf1.plotting.team_color(lap['Team'])
    team_colors.append(color)

Now, we can plot all the data



In [ ]:
fig, ax = plt.subplots()
ax.barh(fastest_laps.index, fastest_laps['LapTimeDelta'],
        color=team_colors, edgecolor='grey')
ax.set_yticks(fastest_laps.index)
ax.set_yticklabels(fastest_laps['Driver'])

# show fastest at the top
ax.invert_yaxis()

# draw vertical lines behind the bars
ax.set_axisbelow(True)
ax.xaxis.grid(True, linestyle='--', color='black', zorder=-1000)

Finally, give the plot a meaningful title



In [ ]:
lap_time_string = strftimedelta(pole_lap['LapTime'], '%m:%s.%ms')

plt.suptitle(f"{quali.weekend.name} {quali.weekend.year} Qualifying\n"
             f"Fastest Lap: {lap_time_string} ({pole_lap['Driver']})")

plt.show()